# Capstone project: Segmenting and Clustering Neighborhoods in Toronto (PART 2)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

## Part 1: Extracting information

First I wanted to verify that I have the right table (and practice another method to upload data), so I copied the table into Excel and then to a csv file. Here I upload the file and use it to see the information

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
from botocore.client import Config
import ibm_boto3

cos = ibm_boto3.client(service_name = "s3",
                      ibm_api_key_id = credentials['IBM_API_KEY_ID'],
                      ibm_service_instance_id = credentials['IAM_SERVICE_ID'],
                        ibm_auth_endpoint = credentials['IBM_AUTH_ENDPOINT'],
                        config = Config(signature_version = "oauth"),
                       endpoint_url = credentials['ENDPOINT']
                      )

In [4]:
cos.download_file(Bucket=credentials['BUCKET'], Key = 'Canada Data.csv', Filename = 'Canada Data.csv')

In [5]:
table_data = pd.read_csv('Canada Data.csv', sep=";")
table_data.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
table_data.shape

(180, 3)

Now that I know how many columns the tables has, I can start using a method to read the table of the Wikipedia page

In [7]:
toronto_data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [8]:
type(toronto_data)

list

In [9]:
len(toronto_data)

3

In [10]:
toronto_df = toronto_data[0]
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [11]:
toronto_df.shape

(180, 3)

Good, the amount of columns and rows is the same as before. Now we can start the pre processing

In [12]:
toronto_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Postal Code    180 non-null    object
 1   Borough        180 non-null    object
 2   Neighbourhood  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB


In [13]:
toronto_df.drop(toronto_df[toronto_df['Borough'] == "Not assigned"].index, inplace = True)
toronto_df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
toronto_df.shape

(103, 3)

In [15]:
(toronto_df['Neighbourhood'].values == 'Not assigned').sum()

0

In [16]:
toronto_df.shape

(103, 3)

## Part 2: Getting location data

In [17]:
# The code was removed by Watson Studio for sharing.

In [18]:
from botocore.client import Config
import ibm_boto3

cos = ibm_boto3.client(service_name = "s3",
                      ibm_api_key_id = credentials_1['IBM_API_KEY_ID'],
                      ibm_service_instance_id = credentials_1['IAM_SERVICE_ID'],
                        ibm_auth_endpoint = credentials_1['IBM_AUTH_ENDPOINT'],
                        config = Config(signature_version = "oauth"),
                       endpoint_url = credentials_1['ENDPOINT']
                      )

In [19]:
cos.download_file(Bucket=credentials_1['BUCKET'], Key = 'Geospatial_Coordinates.csv', Filename = 'Geospatial_Coordinates.csv')

In [21]:
coordinates_data = pd.read_csv('Geospatial_Coordinates.csv')
coordinates_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now that we have the data, lets merge boths data frames

In [23]:
Toronto_df_inner = pd.merge(toronto_df, coordinates_data, on='Postal Code', how='inner')

Toronto_df_inner

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
